# Article Title Generator (V3 - using Firecrawl) 

Summarization use-case in which the user provides an article, which the LLM will analyze to suggest an SEO-optimized title.

**NOTES**:

1. This version supports website scrapping using [Firecrawl](https://www.firecrawl.dev/).<br>
    1. **Note:** There is a Free tier that provides 500 one-time credits (good for scraping 500 pages).
    2. Upon registration, get and add your Firecrawl API Key to the .env file as: **`FIRECRAWL_API_KEY`**.<br><br>
2. Leverage streaming (OpenAI only).<br>
3. The following models were configured:<br>
    1. OpenAI gpt-4o-mini
    2. Llama llama3.2
    3. Deepseek deepseek-r1:1.5b
    4. Firecrawl LLM Extract feature<br><br>
    
   It is possible to configure additional models by adding the new model to the MODELS dictionary and its
   initialization to the CLIENTS dictionary. Then, call the model with --> **`answer =
   get_answer('NEW_MODEL')`**.<br>
4. Users are encouraged to assess and rank the suggested titles using any headline analyzer tool online.
   Example: [ISITWP Headline Analyzer](https://www.isitwp.com/headline-analyzer/). 

In [0]:
# install required libraries if they were not part of the requirements.txt
!pip install firecrawl-py

In [0]:
# confirming Llama is loaded
!ollama pull llama3.2

In [0]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from firecrawl import FirecrawlApp

In [0]:
# set environment variables for OpenAi
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# validate API Key
if not api_key:
    raise ValueError("No OPENAI API Key was found! Please check the .env file.")

In [0]:
# set environment variable for Firecrawl
firecrawl_api_key = os.getenv('FIRECRAWL_API_KEY')

# validate API Key
if not firecrawl_api_key:
    raise ValueError("No FIRECRAWL API Key was found! Please check the .env file.")

In [0]:
# constants
MODELS = { 'GPT': 'gpt-4o-mini', 
          'LLAMA': 'llama3.2', 
          'DEEPSEEK': 'deepseek-r1:1.5b'
         }

CLIENTS = { 'GPT': OpenAI(), 
            'LLAMA': OpenAI(base_url='http://localhost:11434/v1', api_key='ollama'),
            'DEEPSEEK': OpenAI(base_url='http://localhost:11434/v1', api_key='ollama') 
          }

# path to Chrome
# CHROME_PATH = "C:/Program Files/Google/Chrome/Application/chrome.exe"

**Webcrawler** (based on the code from Firecrawl [documentation](https://docs.firecrawl.dev/introduction)).

In [0]:
class WebsiteCrawler:
    def __init__(self, url, wait_time=20, format='markdown'):
        """
        Initialize the WebsiteCrawler using Firecrawl to scrape JavaScript-rendered content.
        """
        self.url = url
        self.wait_time = wait_time
        self.format = format

        try:

            # initialize Firecrawl
            screate_app = FirecrawlApp(api_key=firecrawl_api_key)

            # Scrape a website:
            scrape_result = screate_app.scrape_url(self.url,
                                                  params=self.getParams())
    

            # parse data
            self.title = scrape_result['metadata']['ogTitle']

            # get the content using the appropriate key
            if format == 'markdown':
                # OpenAI, Llama, Deepseek
                self.text = scrape_result['markdown'] 
            elif format == 'json':
                # Firecrawl LLM Extract
                self.text = scrape_result['json']

        except Exception as e:
            print(f"Error occurred: {e}")
            self.title = "Error occurred"
            self.text = ""

    # set appropriate parameters for scraping
    def getParams(self):

      # For OpenAi, Llama or Deepseek
      params={'formats': [self.format], 
              'actions': [{"type": "wait", "milliseconds": self.wait_time}], 
              'includeTags': ["main"], }

      # For Firecrawl LLM extract
      if self.format == 'json':
          params={'formats': [self.format], 
                  'actions': [{"type": "wait", "milliseconds": self.wait_time}], 
                  'jsonOptions': {'systemPrompt': system_prompt, 'prompt': user_prompt, }}
                  
      return params

    # Get Firecrawl LLM extract result
    def getResult(self):

      formated_result = f"""
                        **Optimized Title:** {self.text['Optimized Title']} 
                        <br><br>**Justification:** {self.text['Justification']}
                        """

      # Remove leading and trailing spaces                   
      return formated_result.strip()
            

### Prompts

In [0]:
# system prompt
system_prompt = """
    You are an experienced SEO-focused copywriter. The user will provide an article, and your task is to analyze its content and generate a single, most effective, keyword-optimized title to maximize SEO performance.

Instructions:
Ignore irrelevant content, such as the current title (if any), navigation menus, advertisements, or unrelated text.
Prioritize SEO best practices, considering:
Keyword relevance and search intent (informational, transactional, etc.).
Readability and engagement.
Avoiding keyword stuffing.
Ensure conciseness and clarity, keeping the title under 60 characters when possible for optimal SERP display.
Use a compelling structure that balances informativeness and engagement, leveraging formats like:
Listicles ("10 Best Strategies for…")
How-to guides ("How to Boost…")
Questions ("What Is the Best Way to…")
Power words to enhance click-through rates (e.g., "Proven," "Ultimate," "Essential").
Provide only one single, best title—do not suggest multiple options.
Limit the answer to the following Response Format (Markdown):
Optimized Title: [Provide only one title here]
Justification: [Explain why this title is effective for SEO]

    """

**Provide the article URL and get its content for analysis**

In [0]:
# article url - change to any other article URL
article_url = "https://searchengineland.com/seo-trends-2025-447745"

# get article content
article = WebsiteCrawler(url=article_url)

In [0]:
# user prompt
user_prompt = """
Following the article to be analyzed to suggest a title. Limit the answer to the following Response Format (Markdown): 
Optimized Title: [Provide only one title here]
Justification: [Explain why this title is effective for SEO].
"""

user_prompt = f"{user_prompt} {article}"
                

In [0]:
# message list
messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
           ]

In [0]:
# get suggested title
def get_title(model, **kwargs):
    # stream if GPT
    if 'stream' in kwargs:
        response = CLIENTS[model].chat.completions.create(
                model=MODELS[model],
                messages=messages,
                stream=kwargs['stream']
            )
    else:
        response = CLIENTS[model].chat.completions.create(
                model=MODELS[model],
                messages=messages,
            )

    return response
    

In [0]:
# filter response from model verbose - like Deepseek reasoning/thinking verbose
def filter_response(response):
    filtered_response = response
    # Find last occurrence of 'Optimized Title:' to avoid displaying reasoning verbose
    substring = 'Optimized Title:'
    start = response.rfind(substring)
    if start > -1:
        filtered_response = response[start:]

    # Find if the title has quotation (or other) marks and remove it - this should be improved
    filtered_response = (
        filtered_response.replace('"', '', 2)
        .replace('[', '', 1)
        .replace(']', '', 1)
        .replace('**', '', 2)
    )
     
    return filtered_response

In [0]:
# display suggested title
def display_title(model):
    # get model-suggested title
    title = get_title(model)
    
    display(Markdown(f"### {model} (___{MODELS[model]}___) Answer\n\n_______")) 

    response = ""

    if model == 'GPT':
        display_handle = display(Markdown(""), display_id=True)
        # for chunk in stream:
        for chunk in get_title(model=model, stream=True):
            response += chunk.choices[0].delta.content or ''
            response = (
                response.replace("```","")
                .replace("markdown", "")
                .replace("Optimized Title:", "**Optimized Title:**")
                .replace("Justification:", "**Justification:**")
            )
            update_display(Markdown(response), display_id=display_handle.display_id)
    else:
        response = get_title(model=model)
        response = response.choices[0].message.content
        response = filter_response(response)

        # insert line break to preserve format - only LLAMA
        line_break = "<br><br>"
        if model == "DEEPSEEK":
            line_break = ""
        
        response = (
            response.replace("Optimized Title:", "**Optimized Title:**")
            .replace("Justification:", f"{line_break}**Justification:**") 
        )
        display(Markdown(response))

### Get OpenAI Suggested Title

In [0]:
# get and display openAi suggested title
display_title(model='GPT')

### Get Llama Suggested Title

In [0]:
# get and display Llama suggested title
display_title(model='LLAMA')

### Get Deepseek Suggested Title

In [0]:
# get and display Deepseek title
display_title(model='DEEPSEEK')

### Using Firecrawl LLM Extract (to replace LLMs above - OpenAI, Llama & Deepseek)

### Get Firecrawl Suggested Title

In [0]:
fc_title = WebsiteCrawler(url=article_url, format='json')

display(Markdown(fc_title.getResult()))

### Observations

1. **Firecrawl** is a great alternative to replace both Selenium and BeautifulSoup. However, it is not free.
2. **Firecrawl LLM Extract** feature may replace the calls to other LLMs for analysis and title generation. Note that the result provided seems to be cached upon its first generation. Therefore, the suggested title and its justification will always be the same. 
3. **Deepseek challenges:**\
   a.It always returns its thinking/reasoning verbose, which, while helpful to understand how it works, is not always
      required, such as in this example code. A new function (**filter_response**) was created to remove the additional verbose.\
   b. It is unreliable with the response, sometimes returning the required format for the response instead of the
      actual response. For example, for the title, it may sometimes return:
   
   **Optimized Title:** \[The user wants the suggested title here]
    
### Suggested future improvements

1. Add the logic that would allow each model to assess the recommendations from the different models and 
   select the best among these.
2. Add the logic to leverage an API (if available) that automatically assesses the suggested titles.